# Разведочный анализ данных

Набор данных состоит из 1655 цифровых рентгеновских изображений коленного сустава. Исходные изображения представляют собой 8-битные изображения в оттенках серого. Каждое рентгенологическое рентгеновское изображение коленного сустава вручную классифицировано в соответствии со специальными медицинскими оценками двумя экспертами на 5 классов.

Эксперт I:

1. Normal (515 шт.)
1. Doubtful (478 шт.)
2. Mild (233 шт.)
1. Moderate (222 шт.)
1. Severe (207 шт.)


Эксперт II:

1. Normal (504 шт.)
1. Doubtful (489 шт.)
1. Mild (233 шт.)
1. Moderate (222 шт.)
1. Severe (207 шт.)

Ссылка на данные: https://tnn-hse-medtech.storage.yandexcloud.net/datasets/

## Предварительная настройка

In [4]:
!pip install python-dotenv

In [5]:
%load_ext dotenv
%dotenv

In [6]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import boto3


s3_client = boto3.client(
    's3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
)
BUCKET_NAME = 'tnn-hse-medtech'
DATASET_DIR = 'datasets/'


## Собираем метаданные датасета

In [9]:
import io
import hashlib
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
from ipywidgets import IntProgress
from IPython.display import display


@dataclass()
class Metadata:
    expert: str
    file_id: str
    severity: int
    relative_path: str


def parse_path(key: str) -> Metadata:
    buffer = io.BytesIO()
    s3_client.download_fileobj(BUCKET_NAME, key, buffer)
    data_bytes = buffer.getvalue()
    hash1 = hashlib.sha256(data_bytes).hexdigest()
    hash2 = hashlib.md5(data_bytes).hexdigest()
    path = key.removeprefix(DATASET_DIR)
    expert, severity, _ = path.split('/')
    return Metadata(expert, f'{hash1}:{hash2}', severity, path)

def iterate_by_files():
    paginator = s3_client.get_paginator('list_objects')
    result = paginator.paginate(Bucket=BUCKET_NAME, Prefix=DATASET_DIR)
    for page in result:
        for s3_object in page['Contents']:
            if s3_object['Key'].endswith('.png'):
                yield s3_object['Key']


raw_data = []
files = list(iterate_by_files())
progress = IntProgress(min=0, max=len(files))
display(progress)
with ThreadPoolExecutor(max_workers=50) as pool:
    for data in pool.map(parse_path, files):
        raw_data.append(data)
        progress.value += 1


IntProgress(value=0, max=3300)

# Формирует датасет


In [10]:
data = pd.DataFrame(raw_data)
data


,expert,file_id,severity,relative_path
0,MedicalExpert-I,c7284c66fa8ec0ab4594e5dcd44866f408238f685e999e...,0Normal,MedicalExpert-I/0Normal/NormalG0 (1).png
1,MedicalExpert-I,4a547b94fe02a7565beb21aa9195f4deffe831086da240...,0Normal,MedicalExpert-I/0Normal/NormalG0 (10).png
2,MedicalExpert-I,ca8a296d1e15e0ed84c1ab82440426f28cd0582c19964f...,0Normal,MedicalExpert-I/0Normal/NormalG0 (100).png
3,MedicalExpert-I,6f8bb6bbf0f4def4fdbe1a486064e47eefed0fa832246c...,0Normal,MedicalExpert-I/0Normal/NormalG0 (101).png
4,MedicalExpert-I,d3c8f051ee6c5f59dff4657b7c007860be652e313d85ab...,0Normal,MedicalExpert-I/0Normal/NormalG0 (102).png
...,...,...,...,...
3295,MedicalExpert-II,a762fc9fda75a15538789eb78b53a7ca12631e3c59993d...,4Severe,MedicalExpert-II/4Severe/SevereG4 (95).png
3296,MedicalExpert-II,0ab9e6787dd93e3ad8505c6dcbafa955b91554b026c014...,4Severe,MedicalExpert-II/4Severe/SevereG4 (96).png
3297,MedicalExpert-II,1b3dba34278b1ba238c7051be6886cab2cc2c1c985d368...,4Severe,MedicalExpert-II/4Severe/SevereG4 (97).png
3298,MedicalExpert-II,3338d1a673dd92b49fc1967f0f3cea6bc645fea48bb84b...,4Severe,MedicalExpert-II/4Severe/SevereG4 (98).png


In [ ]:
data.to_csv('raw_data.csv', index=False)


# Нормализуем данные


In [11]:
severity_map = {
    '0Normal': 0,
    '1Doubtful': 1,
    '2Mild': 2,
    '3Moderate': 3,
    '4Severe': 4
}
data['severity'] = data['severity'].map(severity_map)


In [12]:
data.describe(include='object')


,expert,file_id,relative_path
count,3300,3300,3300
unique,2,1633,3300
top,MedicalExpert-I,197ee4a082d0ecd02972532453a8a7aaff300c2348352c...,MedicalExpert-I/0Normal/NormalG0 (1).png
freq,1650,6,1


Видно, что часть хешей совпадает, значит какие-то файлы дублируются в выборке и их можно отбросить. Ищем такие файлы

In [13]:
duplicated_files = data[['file_id', 'relative_path', 'expert']].groupby(['file_id', 'expert'], as_index=False).agg('count')
duplicated_files = duplicated_files[duplicated_files['relative_path'] > 1]
duplicated_files


,file_id,expert,relative_path
310,197ee4a082d0ecd02972532453a8a7aaff300c2348352c...,MedicalExpert-I,3
311,197ee4a082d0ecd02972532453a8a7aaff300c2348352c...,MedicalExpert-II,3
364,1dcec59295c228968cfbe39c1e16028b25be68ea58c78b...,MedicalExpert-I,2
365,1dcec59295c228968cfbe39c1e16028b25be68ea58c78b...,MedicalExpert-II,2
898,451842e164a05a07973360035fceb73cb9b463eb5fe1c1...,MedicalExpert-I,2
899,451842e164a05a07973360035fceb73cb9b463eb5fe1c1...,MedicalExpert-II,2
1120,54cc7904e251af243e3033f895997e37a6976f4e5b78cb...,MedicalExpert-I,2
1121,54cc7904e251af243e3033f895997e37a6976f4e5b78cb...,MedicalExpert-II,2
1238,5c78a76c95d62132765bf5337c8ec0aab4bcf32d6dd97c...,MedicalExpert-I,2
1239,5c78a76c95d62132765bf5337c8ec0aab4bcf32d6dd97c...,MedicalExpert-II,2


Проверяем файлы

In [ ]:
from PIL import Image

paths = data[data['file_id'] == '197ee4a082d0ecd02972532453a8a7aaff300c2348352c866a1dba15cbb2554d:f0dc21fa5616728d830149fd3d0513bd']['relative_path']
for key in paths:
    buffer = io.BytesIO()
    s3_client.download_fileobj(BUCKET_NAME, f'{DATASET_DIR}{key}', buffer)
    buffer.seek(0)
    image = Image.open(buffer)
    print(key)
    display(image)


Видно, что изображения идентичные. Проверяем, есть ли разные оценки для таких изображений от каждого эксперта в отдельности

In [14]:
duplicated_data = data[data['file_id'].isin(duplicated_files['file_id'])]
duplicated_data[['expert', 'severity', 'file_id']].groupby(['expert', 'file_id'], as_index=False).agg(['nunique', 'first'])


expert                                            file_id  \
                                                                          
0    MedicalExpert-I  197ee4a082d0ecd02972532453a8a7aaff300c2348352c...   
1    MedicalExpert-I  1dcec59295c228968cfbe39c1e16028b25be68ea58c78b...   
2    MedicalExpert-I  451842e164a05a07973360035fceb73cb9b463eb5fe1c1...   
3    MedicalExpert-I  54cc7904e251af243e3033f895997e37a6976f4e5b78cb...   
4    MedicalExpert-I  5c78a76c95d62132765bf5337c8ec0aab4bcf32d6dd97c...   
5    MedicalExpert-I  7a21c7e7f35d1b9cd50cd2bb50fb3c5893fe51806e3127...   
6    MedicalExpert-I  a2038be623857565ab04f5d050b49fad1cb9560abff33b...   
7    MedicalExpert-I  a584c0ac57356d4416e8501679c8eaea18afd71d69c03f...   
8    MedicalExpert-I  ae12756e91aa76578235cd53766a5c74991d2266589308...   
9    MedicalExpert-I  b1448f0ce641f512a7eb75bf195739bcd41517add88165...   
10   MedicalExpert-I  b1691371b73e79e3936f7aac9adf389a24c594adc056e4...   
11   MedicalExpert-I  b77349ed8bc71862a59379ef157bccb6381b74a077025b...   
12   MedicalExpert-I  d22bd4c1f75758819b1746090725b5338d3193631bdc6b...   
13   MedicalExpert-I  d97f89d4e1826abb55e1fa5868e27ee80bb3a8f0c8a86d...   
14   MedicalExpert-I  f96a0d0a156b931634ded8b4d77e0e0f1a99a2a1a25fa6...   
15  MedicalExpert-II  197ee4a082d0ecd02972532453a8a7aaff300c2348352c...   
16  MedicalExpert-II  1dcec59295c228968cfbe39c1e16028b25be68ea58c78b...   
17  MedicalExpert-II  451842e164a05a07973360035fceb73cb9b463eb5fe1c1...   
18  MedicalExpert-II  54cc7904e251af243e3033f895997e37a6976f4e5b78cb...   
19  MedicalExpert-II  5c78a76c95d62132765bf5337c8ec0aab4bcf32d6dd97c...   
20  MedicalExpert-II  7a21c7e7f35d1b9cd50cd2bb50fb3c5893fe51806e3127...   
21  MedicalExpert-II  a2038be623857565ab04f5d050b49fad1cb9560abff33b...   
22  MedicalExpert-II  a584c0ac57356d4416e8501679c8eaea18afd71d69c03f...   
23  MedicalExpert-II  ae12756e91aa76578235cd53766a5c74991d2266589308...   
24  MedicalExpert-II  b1448f0ce641f512a7eb75bf195739bcd41517add88165...   
25  MedicalExpert-II  b1691371b73e79e3936f7aac9adf389a24c594adc056e4...   
26  MedicalExpert-II  b77349ed8bc71862a59379ef157bccb6381b74a077025b...   
27  MedicalExpert-II  d22bd4c1f75758819b1746090725b5338d3193631bdc6b...   
28  MedicalExpert-II  d97f89d4e1826abb55e1fa5868e27ee80bb3a8f0c8a86d...   
29  MedicalExpert-II  f96a0d0a156b931634ded8b4d77e0e0f1a99a2a1a25fa6...   

   severity        
    nunique first  
0         1     3  
1         1     1  
2         1     3  
3         1     3  
4         1     2  
5         1     3  
6         1     1  
7         1     0  
8         1     2  
9         1     2  
10        1     2  
11        1     2  
12        1     3  
13        1     1  
14        1     3  
15        1     3  
16        1     1  
17        1     3  
18        1     3  
19        1     2  
20        1     3  
21        1     1  
22        1     0  
23        1     2  
24        1     2  
25        1     2  
26        1     2  
27        1     3  
28        1     1  
29        1     3

Оценки совпадают, значит можно откинуть дубликаты

In [15]:
deleted_paths = data[data.duplicated(['expert', 'file_id'], keep='first')]['relative_path']
deleted_paths


216          MedicalExpert-I/0Normal/NormalG0 (294).png
832      MedicalExpert-I/1Doubtful/DoubtfulG1 (386).png
907      MedicalExpert-I/1Doubtful/DoubtfulG1 (453).png
965       MedicalExpert-I/1Doubtful/DoubtfulG1 (76).png
1004             MedicalExpert-I/2Mild/MildG2 (110).png
1116             MedicalExpert-I/2Mild/MildG2 (211).png
1118             MedicalExpert-I/2Mild/MildG2 (213).png
1195              MedicalExpert-I/2Mild/MildG2 (74).png
1200              MedicalExpert-I/2Mild/MildG2 (79).png
1342     MedicalExpert-I/3Moderate/ModerateG3 (206).png
1343     MedicalExpert-I/3Moderate/ModerateG3 (207).png
1344     MedicalExpert-I/3Moderate/ModerateG3 (208).png
1347     MedicalExpert-I/3Moderate/ModerateG3 (210).png
1354     MedicalExpert-I/3Moderate/ModerateG3 (217).png
1385      MedicalExpert-I/3Moderate/ModerateG3 (46).png
1387      MedicalExpert-I/3Moderate/ModerateG3 (48).png
1432      MedicalExpert-I/3Moderate/ModerateG3 (89).png
1866        MedicalExpert-II/0Normal/NormalG0 (2

Сохраняем в файл для дальнейшего использования при фильтрации

In [ ]:
deleted_paths.to_csv('deleted_paths.csv', index=False)


In [16]:
unique_data = data.drop_duplicates(['expert', 'file_id'], keep='first')
unique_data


,expert,file_id,severity,relative_path
0,MedicalExpert-I,c7284c66fa8ec0ab4594e5dcd44866f408238f685e999e...,0,MedicalExpert-I/0Normal/NormalG0 (1).png
1,MedicalExpert-I,4a547b94fe02a7565beb21aa9195f4deffe831086da240...,0,MedicalExpert-I/0Normal/NormalG0 (10).png
2,MedicalExpert-I,ca8a296d1e15e0ed84c1ab82440426f28cd0582c19964f...,0,MedicalExpert-I/0Normal/NormalG0 (100).png
3,MedicalExpert-I,6f8bb6bbf0f4def4fdbe1a486064e47eefed0fa832246c...,0,MedicalExpert-I/0Normal/NormalG0 (101).png
4,MedicalExpert-I,d3c8f051ee6c5f59dff4657b7c007860be652e313d85ab...,0,MedicalExpert-I/0Normal/NormalG0 (102).png
...,...,...,...,...
3295,MedicalExpert-II,a762fc9fda75a15538789eb78b53a7ca12631e3c59993d...,4,MedicalExpert-II/4Severe/SevereG4 (95).png
3296,MedicalExpert-II,0ab9e6787dd93e3ad8505c6dcbafa955b91554b026c014...,4,MedicalExpert-II/4Severe/SevereG4 (96).png
3297,MedicalExpert-II,1b3dba34278b1ba238c7051be6886cab2cc2c1c985d368...,4,MedicalExpert-II/4Severe/SevereG4 (97).png
3298,MedicalExpert-II,3338d1a673dd92b49fc1967f0f3cea6bc645fea48bb84b...,4,MedicalExpert-II/4Severe/SevereG4 (98).png


## Анализ нормализованных данных

Для начала посмотрим как сильно расходятся оценки у экспертов по файлам

In [17]:
cross_check = unique_data.pivot(
    index='file_id', columns='expert', values='severity'
    ).reset_index()
cross_check


expert,file_id,MedicalExpert-I,MedicalExpert-II
0,000555ee2250db28d7ccb076cd8ed02dc46ad149255231...,0,0
1,001a4766d9a9cc32c1d7bc65b6a6d6fa6f18a5540f576e...,3,3
2,00439f2700963bfc89ab72a4fe7711299228d8db3fd4cb...,2,2
3,005c7b80497f04a63b685eafe6797d672754684a9317ac...,2,2
4,00668822b6f82a52fd186a0d8583f3e276bab809b32b71...,1,1
...,...,...,...
1628,fe910f9f5eea10fe03f6b8dbc10c4a318b55053aaa580b...,3,3
1629,fecd647553be61596154b3766dbe39a5d3b2b9fb2edede...,3,3
1630,ff49383c140e3cd433c0ed3d52c923378a7958482dd4e2...,1,1
1631,ff4bbd6aa27f3cf0e5d4ad3e2e83c025c16a8b372a82a0...,0,0


In [18]:
cross_check.groupby(
        ['MedicalExpert-I', 'MedicalExpert-II']
        ).agg('count')


expert                            file_id
MedicalExpert-I MedicalExpert-II         
0               0                     502
                1                      11
1               1                     474
2               2                     227
3               3                     213
4               4                     206

В основном оценки совпадают, но обнаружены 11 файлов, которые были оценены по разному

In [19]:
different_level = cross_check[(cross_check['MedicalExpert-II']!=cross_check['MedicalExpert-I'])]
different_level


expert,file_id,MedicalExpert-I,MedicalExpert-II
9,0184a61fb664048be62beaebd18350091b1da331469dc3...,0,1
250,283fb711f3d2f6af28b92ef4ab54c2ab98b7537fc7b278...,0,1
334,32dc600ef010e8251457ab912c242eb9f288b208059606...,0,1
361,37e674b83d1828aa25f4a370c499b691715bcb100485f0...,0,1
620,5cd936fc2c88c431ce72ab9f3e5cf2907e5827136318ba...,0,1
649,6144aa47b5c8f9d99ae704e0562a2d361ab9da5c13768e...,0,1
716,6b96ee74963e6cb9f3c6823cd5f563d31a032a63b843bb...,0,1
1183,b56d832cd973346c6504f2fb56ba673ac1c1bb49a6d064...,0,1
1195,b7ed43175bbd8d7a842ed60ce405c0119187c1faff8f7a...,0,1
1420,df1fb9f2f43783f68d2ae8de7052b2a1a9017fa9003442...,0,1


Убедимся, что изображения идентичные

In [ ]:
different_severity_paths = unique_data[unique_data['file_id']==different_level['file_id'].iloc[0]]['relative_path']
for key in different_severity_paths:
    buffer = io.BytesIO()
    s3_client.download_fileobj(BUCKET_NAME, f'{DATASET_DIR}{key}', buffer)
    buffer.seek(0)
    image = Image.open(buffer)
    print(key)
    display(image)


Сохраним пути таких файлов

In [ ]:
data[data['file_id'].isin(different_level['file_id'])][['file_id', 'severity', 'relative_path']].to_csv('different_severity.csv', index=False)


In [22]:
normalized_data = data[~data['file_id'].isin(different_level['file_id'])]
normalized_data.to_csv('normalized_data.csv', index=False)
